In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import os
import numpy as np
from PIL import Image



# Function to load and convert image to pixel data
def image_to_pixels(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')  # Ensure the image is in RGB mode
    img_array = np.array(img)  # Convert image to NumPy array
    img_array_flattened = img_array.flatten()  # Flatten the array
    return img_array_flattened.tolist()  # Convert flattened array to a list


def loadBeansDataset(imagesLimit = False):
    # Loop through each image in the directory
    data_directory = 'dataset beans/train/'
    imagesData = { "images": [], "labels" : [] }
    labels_data = ['angular_leaf_spot', 'bean_rust', 'healthy']
    for label in labels_data:
        directory = data_directory + label
        if(imagesLimit) :
            filenames = os.listdir(directory)[:imagesLimit]
        else :
            filenames = os.listdir(directory)
        for filename in filenames:
            if filename.endswith('.jpg'): # Adjust file extensions as needed
                img_path = os.path.join(directory, filename)
                pixel_row = image_to_pixels(img_path)
                imagesData["images"].append(pixel_row)
                imagesData["labels"].append(label)
    return imagesData


# Load beans dataset (replace with your image dataset)
beansDataset = loadBeansDataset(50)
X, y = beansDataset["images"], beansDataset["labels"]
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize kNN classifier
knn = KNeighborsClassifier(n_neighbors=5)

# Train the classifier
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)